In [1]:
import numpy as np

In [8]:
fn = '/'.join(['e:', 'Data', 'zebtrastack_temp', 'NIH_Cxr8', 
               '128x128', 'channelized_clahe_processed', 
               '00000032_009.npy'])
sample = np.load(fn)
sample.dtype

dtype('float32')

In [10]:
import pathlib
import tensorflow as tf
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='flower_photos', 
                                   untar=True)
data_dir = pathlib.Path(data_dir)

228818944/228813984 [==============================] - 134s 1us/step


In [11]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3670


In [12]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [13]:
for f in list_ds.take(5):
  print(f.numpy())

b'C:\\Users\\test\\.keras\\datasets\\flower_photos\\sunflowers\\5830614551_e460a1215c.jpg'
b'C:\\Users\\test\\.keras\\datasets\\flower_photos\\tulips\\7136973281_b2a935ce20.jpg'
b'C:\\Users\\test\\.keras\\datasets\\flower_photos\\sunflowers\\16975010069_7afd290657_m.jpg'
b'C:\\Users\\test\\.keras\\datasets\\flower_photos\\dandelion\\3533075436_0954145b9f_m.jpg'
b'C:\\Users\\test\\.keras\\datasets\\flower_photos\\dandelion\\15358221063_2c6e548e84.jpg'


In [14]:
class_names = np.array(sorted([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"]))
print(class_names)

['daisy' 'dandelion' 'roses' 'sunflowers' 'tulips']


In [63]:
val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

In [64]:
print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())

2936
734


In [46]:
for d in train_ds.take(5):
    print(d)

tf.Tensor(b'C:\\Users\\test\\.keras\\datasets\\flower_photos\\dandelion\\3502447188_ab4a5055ac_m.jpg', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\test\\.keras\\datasets\\flower_photos\\sunflowers\\200557977_bf24d9550b.jpg', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\test\\.keras\\datasets\\flower_photos\\roses\\4504220673_af754fcb40_n.jpg', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\test\\.keras\\datasets\\flower_photos\\sunflowers\\6141150299_b46a64e4de.jpg', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\test\\.keras\\datasets\\flower_photos\\tulips\\17012955700_7141d29eee.jpg', shape=(), dtype=string)


In [65]:
import os
def get_label(file_path):
    print(f"get_label({file_path})")
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    print(parts)
    # The second to last is the class-directory
    # one_hot = parts[-2] == class_names
    # Integer encode the label
    return parts[-2] #tf.argmax(one_hot)

In [66]:
img_height, img_width = 256, 256
def decode_img(img):
    print(img.shape)
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # resize the image to the desired size
    return tf.image.resize(img, [img_height, img_width])

In [67]:
def process_path(file_path):
    print(file_path)
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [68]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_ds = train_ds.map(process_path)
val_ds = val_ds.map(process_path)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 3, expecting 4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 3, expecting 4
Tensor("args_0:0", shape=(), dtype=string)
get_label(Tensor("args_0:0", shape=(), dtype=string))
Tensor("StringSplit/RaggedGetItem/strided_slice_5:0", shape=(None,), dtype=string)
()
Tensor("args_0:0", shape=(), dtype=string)
get_label(Tensor("args_0:0", shape=(), dtype=string))
Tensor("StringSplit/RaggedGetItem/strided_slice_5:0", shape=(None,), dtype=string)
()


In [69]:
for image, label in train_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (256, 256, 3)
Label:  b'dandelion'
